In [36]:
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display



### Direct HTTP Call to locally running ollama api on port number 11434 as follows


#### Steps cell wise

##### Step 1:- define the api key 
##### Step 2:- define the system Prompt and the user prompt
##### Step 3:- Define and initialize the payload
##### Step 4:- Veify if/which ollama model you want to use using the ollama model
##### Step 5:- Store the respone of the request.post(ollama_api, json=payload, headers=HEADERS) and then display it


In [37]:
OLLAMA_API = "http:''localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json" }
MODEL = "gemma3:4b"

In [38]:
messages = [
    {"role":"user", "content":"Tell me something about quantum physics that very interesting facts."}
]

In [39]:
payload = {
    "model": MODEL,
    "messages": messages,
    "stream": False
}

#### Using the ollama package:- now the same thing as above but using the ollama package instead of direct HTTP call
#### Under the hood its making the same call as above to the ollama server running at localhost:11434

In [40]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Okay, let's dive into some fascinating and mind-bending facts about quantum physics! Here are a few that consistently blow people's minds, categorized for a bit of clarity:

**1. The Weirdness of Observation & Wave-Particle Duality:**

* **The Observer Effect:** This is arguably the most famous and unsettling quantum concept. It states that *the act of observing a quantum system fundamentally changes it*.  Think of it like this:  Before you measure the position of an electron, it exists in a blurry, probabilistic state – a superposition of all possible positions.  The moment you try to pinpoint its location, the wave function "collapses," and the electron "chooses" one location. It's not that the electron *didn't* have a position before, but the act of measurement forces it to manifest in a specific place.
* **Wave-Particle Duality:** Quantum objects like electrons and photons don’t behave as either waves *or* particles. They exhibit both properties, depending on how you observe them. 

##### Using the OpenAI API for the same summary task instead of ollama package

In [41]:
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model = MODEL,
    messages = messages
)

print(response.choices[0].message.content)

Okay, let's dive into some fascinating and mind-bending facts from the world of quantum physics! Here are a few that consistently blow people's minds:

**1. Superposition: Existing in Multiple States at Once**

* **The Idea:** This is arguably the most famous and weirdest concept. Before you measure a quantum particle (like an electron), it doesn't have a definite position, momentum, or spin. Instead, it exists in a *superposition* of all possible states simultaneously.  
* **Think of it like this:** Imagine a coin spinning in the air. It's neither heads nor tails until it lands. Similarly, an electron can be in multiple locations or spin directions at the same time.
* **The Measurement Problem:** The really strange thing is that *the act of measuring* forces the particle to "choose" one of those states. It's like the coin finally landing on heads or tails.


**2. Quantum Entanglement: Spooky Action at a Distance**

* **The Idea:** Two entangled particles become linked, regardless of t

##### The above task can be completed using the openai api even if the model was ollama i.e. open source because OpenAI api is simply a code written by OpenAI engineers that makes calls over the internet to an endpoint.

###### openeai.chat.completions.create() python code just makes a web request to the "https://api.openai.com/v1/chat/completions"  so instead of this we just place our own localhost://111434/api/chat and it works becuase  of the same format structure.

## Website Summarizer Exercise

In [42]:
class Website:
    def __init__(self,url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script","style","img","input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)




In [59]:
sec = "https://www.satyasai.co.in"
sec1 = "https://www.satyasai.co.in/about-us"
# print(sec.title)
# print(sec.text)

#### Now we will use the below prompts to communicate with the local ollama gemma3:4b llms

In [60]:
system_prompt = "You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown."

In [61]:

def user_prompt_for(website):
    user_prompt = f"You  are looking at a website {website.title}"
    user_prompt += "\nThe contentes of this website is as follows; "
    user_prompt += website.text
    return user_prompt

In [62]:
print(user_prompt_for(sec))

AttributeError: 'str' object has no attribute 'text'

In [63]:
def messages_for(website):
    return [
        {"role":"system", "content":system_prompt},
        {"role":"user", "content":user_prompt_for(website)}
    ]

In [64]:
messages_for(sec)

AttributeError: 'str' object has no attribute 'text'

In [65]:
def summarize(url):
    website = Website(url)
    # ollama_via_openai = OpenAI(base_url="https://localhost:11434/v1")
    response = ollama.chat(
        model=MODEL,
        messages=messages_for(website)
    )

    return response["message"]["content"]

In [66]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [67]:
display_summary(sec)

Satyasai Construction is a materials handling construction company with over 15 years of experience. They offer solutions including:

*   **Materials Handling:** Focusing on loading, unloading, movement, and management of materials and equipment.
*   **On-Site Material Management:**  Streamlining resource delivery and management.
*   **Equipment Handling:**  Installation and maintenance of materials handling equipment.
*   **Waste Removal and Recycling:**  Managing waste disposal.

They provide services like facility design & construction, conveyor systems & automation, equipment installation & maintenance, and system upgrades.  The company has a team of 450 active experts and has worked with clients like Indian Oil Corporation Limited, Engineers India Limited, and National Aluminium Corporation Limited.  They emphasize reliable and efficient solutions for the construction industry.

In [68]:
display_summary(sec1)

Satyasai Construction specializes in materials and warehouse handling construction solutions. They offer facility design & construction, conveyor systems & automation, equipment installation & maintenance, and system upgrades/retrofitting. The company has 15+ years of experience and focuses on the movement, transportation, and management of materials and equipment within construction sites. They operate out of offices in Odisha and West Bengal.